In [1]:

import pandas as pd
import logging
import traceback
import json
from typing import Dict, List, Optional, Any
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from graph.perfil.state import (EstadoAnalisisPerfil, PerfilUsuario, InfoPasajeros, FiltrosInferidos, EconomiaUsuario, InfoClimaUsuario)
from utils.enums import (Transmision, NivelAventura, TipoUsoProfesional,  DimensionProblematica, EstiloConduccion, TipoMecanica, FrecuenciaUso, DistanciaTrayecto, FrecuenciaViajesLargos)

test_config = {"configurable": { "thread_id": "test_thread_notebook_1234"}}
from graph.perfil.nodes import (
    calcular_recomendacion_economia_modo1_node, # Asumimos que existe pero no lo usaremos directamente si el modo es 2
   aplicar_filtros_pasajeros_node, aplicar_filtros_pasajeros_node,construir_filtros_node, calcular_flags_dinamicos_node,calcular_pesos_finales_node, formatear_tabla_resumen_node, calcular_km_anuales_postprocessing_node, buscar_coches_finales_node)

try:
    from config.llm import llm, llm_explicacion_coche
except ImportError:
    print("WARN: No se pudo importar la configuración de LLM. Las explicaciones podrían no funcionar.")
    llm_explicacion_coche = None

# Configurar logging básico para ver los DEBUG de los nodos
logging.getLogger().setLevel(level=logging.DEBUG)

logging.basicConfig(level=logging.DEBUG) #o level=logging.DEBUG

# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 150)


Entorno local detectado. Verificando GOOGLE_APPLICATION_CREDENTIALS del archivo .env...


In [2]:

PERFIL_A_PROBAR = "PERFIL" 
print(f"--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: {PERFIL_A_PROBAR} ---")

if PERFIL_A_PROBAR == "PERFIL":
    test_info_clima = InfoClimaUsuario(
        MUNICIPIO_ZBE=False, ZONA_LLUVIAS=False, ZONA_NIEBLAS=False, ZONA_NIEVE=False,
        ZONA_CLIMA_MONTA=False, ZONA_GLP=False, ZONA_GNV=False, cp_valido_encontrado=True
    )
    test_preferencias_usuario = PerfilUsuario(
        apasionado_motor="si", #activa premium y singular
        valora_estetica="si",  #activa estetica
        prefiere_diseno_exclusivo="si", #activa singular
        circula_principalmente_ciudad = 'si',
        uso_profesional="no", 
        tipo_uso_profesional=None, #TipoUsoProfesional.PASAJEROS/CARGA/MIXTO
        altura_mayor_190="no",
        coche_principal_hogar="si",
        rating_fiabilidad_durabilidad=9,
        rating_seguridad=5,
        rating_comodidad=5,
        rating_impacto_ambiental=5,
        rating_costes_uso=5, 
        rating_tecnologia_conectividad=5,
        transporta_carga_voluminosa="si", 
        necesita_espacio_objetos_especiales= "no",
        arrastra_remolque="no", 
        tiene_garage="si", 
        espacio_sobra_garage = "si",  #si/no 
        #problema_dimension_garage= [DimensionProblematica.LARGO , DimensionProblematica.ANCHO],
        problemas_aparcar_calle= "no", 
        tiene_punto_carga_propio="no",
        solo_electricos="no", 
        prioriza_baja_depreciacion="no",
        frecuencia_uso=FrecuenciaUso.FRECUENTEMENTE,
        distancia_trayecto=DistanciaTrayecto.ENTRE_10_Y_50_KM , 
        realiza_viajes_largos = 'si',
        frecuencia_viajes_largos= FrecuenciaViajesLargos.FRECUENTEMENTE, #FrecuenciaViajesLargos.OCASIONALMENTE,
        transmision_preferida=Transmision.AMBOS,
        estilo_conduccion=EstiloConduccion.DEPORTIVO,
        aventura=NivelAventura.ninguna,
        
    ) ##Si suele_llevar_acompanantes= false: info_pasajeros.frecuencia a "nunca".["nunca", "ocasional", "frecuente"]
    test_info_pasajeros = InfoPasajeros(suele_llevar_acompanantes= False ,
                                frecuencia_viaje_con_acompanantes = None, #ocasional , frecuente
                                num_ninos_silla=0, num_otros_pasajeros=0) 
    
    
    # test_economia_usuario = EconomiaUsuario(
    # presupuesto_definido=True,  # <-- El equivalente a FALSE=>'modo=1'asesoramiento,
    # ingresos=None,
    # ahorro=None,
    # pago_contado=10000.0,
    # cuota_max=None,
    # entrada=None,
    # anos_posesion=None
    # ) #26.900 al contado

    
    test_economia_usuario = EconomiaUsuario(
    presupuesto_definido=False,  # <-- El equivalente a 'modo=1'asesoramiento,
    ingresos=50000.0,
    ahorro=50000.0,
    pago_contado=None,
    cuota_max=None,
    entrada=None,
    #anos_posesion=8
    ) #26.900 al contado


# 
initial_filtros_inferidos =  FiltrosInferidos( 
     #tipo_mecanica=[TipoMecanica.GASOLINA, TipoMecanica.DIESEL, TipoMecanica.GLP , TipoMecanica.GNV , TipoMecanica.HEVD , TipoMecanica.HEVG , TipoMecanica.PHEVG , TipoMecanica.PHEVD ] # Ejemplo
)

initial_state_dict: EstadoAnalisisPerfil = {
    "messages": [AIMessage(content="...")], 
    "info_clima_usuario": test_info_clima,
    "preferencias_usuario": test_preferencias_usuario, 
    "info_pasajeros": test_info_pasajeros,
    "economia": test_economia_usuario, 
    "filtros_inferidos": initial_filtros_inferidos,
    "pesos": None, "tabla_resumen_criterios": None, "coches_recomendados": None,
    "pregunta_pendiente": None, "penalizar_puertas_bajas": False, "priorizar_ancho": False,
    # Flags que se calcularán en calcular_flags_dinamicos_node
    "flag_penalizar_low_cost_comodidad": False, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_deportividad_comodidad": False, #Activando flags penalización comodidad flag penalizar depor comod y flag penalizar lowcost comod
    "flag_penalizar_antiguo_por_tecnologia": False, # Lógica para Flag de Penalización por Antigüedad y Tecnología > 7
    "aplicar_logica_distintivo_ambiental": False,
    "es_municipio_zbe": False,
    "codigo_postal_extraido_temporal": None,
    "tipo_mensaje_cp_llm": None,
    "_decision_cp_validation": None,
    "penalizar_bev_reev_aventura_ocasional": False,
    "penalizar_phev_aventura_ocasional": False,
    "penalizar_electrificados_aventura_extrema": False,
    "favorecer_carroceria_montana": False,
    'favorecer_carroceria_comercial': False,
    'favorecer_carroceria_pasajeros_pro': False,
    'desfavorecer_carroceria_no_aventura': False, #Aventura 'ninguna' y no es clima de montaña. Activando flag para desfavorecer PICKUP/TODOTERRENO."
    'favorecer_suv_aventura_ocasional': False,
    "favorecer_pickup_todoterreno_aventura_extrema": False,
    "aplicar_logica_objetos_especiales": False,
    "favorecer_carroceria_confort": False,
    "flag_logica_uso_ocasional": False,
    "flag_favorecer_bev_uso_definido": False,
    "flag_penalizar_phev_uso_intensivo":False,
    "flag_favorecer_electrificados_por_punto_carga": False,  
    'penalizar_awd_ninguna_aventura' : False,
    'flag_bonus_awd_clima_adverso' : False,
    'favorecer_awd_aventura_ocasional' : False,
    'favorecer_awd_aventura_extrema' : False,
    'flag_logica_reductoras_aventura': False,
    'flag_logica_diesel_ciudad' : False,
    'flag_bonus_fiab_dur_critico': False,
    'flag_bonus_fiab_dur_fuerte': False,
    'flag_bonus_costes_critico': False
    
}
# --- 3. Ejecutar la Secuencia de Nodos de Finalización y Búsqueda (SECUENCIA CORREGIDA) ---
print("\n--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---")
current_state = initial_state_dict.copy()

try: 
    # --- PASO 1: Lógica Económica (Establece el presupuesto en los filtros) ---
    print("\n1. Calculando recomendación económica (Modo Asesoramiento)...")
    output_economia = calcular_recomendacion_economia_modo1_node(current_state)
    current_state.update(output_economia)
    # En este punto, `filtros_inferidos` ya contiene la cuota_max_calculada.
    print(f"✅ Filtros después de cálculo económico: {current_state.get('filtros_inferidos')}")

    # --- PASO 2: Lógica de Pasajeros y Filtros Adicionales ---
    # Estos nodos AÑADEN información al objeto de filtros existente.
    
    print("\n2. Aplicando filtros de pasajeros (plazas_min)...")
    # Nota: Este nodo ya debería haberse ejecutado en la etapa de Pasajeros.
    # Es posible que no necesites llamarlo de nuevo aquí si el estado inicial ya es correcto.
    output_filtros_pasajeros = aplicar_filtros_pasajeros_node(current_state)
    current_state.update(output_filtros_pasajeros)

    print("\n3. Construyendo filtros de mecánica (ZBE, etc.)...")
    output_filtros_mecanica = construir_filtros_node(current_state)
    current_state.update(output_filtros_mecanica)
    print(f"✅ Filtros después de añadir mecánica: {current_state.get('filtros_inferidos')}")

    # --- PASO 3: Post-procesamiento y Cálculos Finales ---
    # Estos nodos leen el estado completo y calculan los valores finales.
    
    print("\n4. Calculando KM anuales...")
    output_km_anuales = calcular_km_anuales_postprocessing_node(current_state)
    current_state.update(output_km_anuales)
    print(f"✅ KM Anuales Estimados: {current_state.get('km_anuales_estimados')}")
    
    print("\n5. Calculando Flags dinámicos...")
    output_flags = calcular_flags_dinamicos_node(current_state)
    current_state.update(output_flags)

    print("\n6. Calculando Pesos finales...")
    output_pesos = calcular_pesos_finales_node(current_state)
    current_state.update(output_pesos)

    print("\n7. Generando Tabla Resumen de Criterios...")
    output_tabla = formatear_tabla_resumen_node(current_state)
    current_state.update(output_tabla)
    
    # --- PASO 4: Búsqueda Final ---
    print("\n8. 🚀 Ejecutando Búsqueda Final en BigQuery...")
    output_final_busqueda = buscar_coches_finales_node(current_state, test_config)
    current_state.update(output_final_busqueda)
    
    # --- ANÁLISIS DEL RESULTADO FINAL ---
    # (Tu excelente lógica de análisis con Pandas se mantiene igual)
    print("\n\n--- 🏁 ANÁLISIS DEL RESULTADO FINAL ---")
    
    final_message = current_state['messages'][-1] if current_state.get('messages') else None

    if isinstance(final_message, AIMessage):
        print("\n--- Mensaje de Texto (lo que ve el usuario) ---")
        final_message.pretty_print()

        if final_message.additional_kwargs and 'payload' in final_message.additional_kwargs:
            print("\n--- 🕵️ Datos Estructurados (lo que procesa el frontend) 🕵️ ---")
            payload = final_message.additional_kwargs['payload']
            
            if payload and payload.get('type') == 'car_recommendation':
                coches_recomendados = payload.get('cars', [])
                if coches_recomendados:
                    print(f"\nSe encontraron {len(coches_recomendados)} coches en el payload:")
                    df = pd.DataFrame(coches_recomendados)
                    display(df)
                else:
                    print("⚠️ El payload no contenía coches.")
            else:
                print("⚠️ El payload no es del tipo 'car_recommendation' o está vacío.")
        else:
            print("\n--- No se encontraron datos estructurados (¡Correcto si no hay coches!). ---")
    else:
        print("❌ No se generó ningún mensaje final del agente.")

except Exception as e_test:
    print(f"\n❌ ERROR durante la prueba de la secuencia de nodos: {e_test}")
    traceback.print_exc()

    
 

INFO:root:--- Ejecutando Nodo: calcular_recomendacion_economia_modo1_node ---
INFO:root:DEBUG (CalcEconModo1) ► Modo 'Asesoramiento' detectado. Iniciando cálculo...
INFO:root:--------------------------------------------------
INFO:root: INSPECCIÓN DE CÁLCULO ECONÓMICO (MODO ASESORAMIENTO) 
INFO:root:  - Inputs: Ingresos=50000.0€, Ahorro=50000.0€, Plazo (fijo)=8 años
INFO:root:  - Ahorro Utilizable (75%): 37,500.00€
INFO:root:  - Gasto Potencial Total (a 8 años): 38,000.00€
INFO:root:  - Capacidad Ahorro Mensual (Cuota Máx. teórica): 395.83€/mes
INFO:root:  - Condición Contado: ¿38,000.00€ <= 37,500.00€? -> False
INFO:root:--------------------------------------------------
INFO:root:✅ RECOMENDACIÓN FINAL: Modo=Financiado, Precio Máx.=None, Cuota Máx.=395.8333333333333
INFO:root:--- Ejecutando Nodo: aplicar_filtros_pasajeros_node ---
DEBUG:root:DEBUG (Aplicar Filtros Pasajeros) ► Info recibida: freq='None', X=0, Z=0
DEBUG:root:DEBUG (Aplicar Filtros Pasajeros) ► Calculado plazas_min = 1


--- 🚗 CONFIGURANDO PRUEBA PARA PERFIL: PERFIL ---

--- 🚀 INICIANDO SECUENCIA DE FINALIZACIÓN Y BÚSQUEDA ---

1. Calculando recomendación económica (Modo Asesoramiento)...
✅ Filtros después de cálculo económico: tipo_mecanica=None tipo_carroceria=None modo_adquisicion_recomendado='Financiado' precio_max_contado_recomendado=None cuota_max_calculada=395.8333333333333 plazas_min=None

2. Aplicando filtros de pasajeros (plazas_min)...

3. Construyendo filtros de mecánica (ZBE, etc.)...
--- Ejecutando Nodo: construir_filtros_node ---
DEBUG (Filtros) ► Preferencias e info_clima disponibles. Construyendo filtros...
DEBUG (Filtros) ► Filtros finales construidos: tipo_mecanica=[<TipoMecanica.BEV: 'BEV'>, <TipoMecanica.DIESEL: 'DIESEL'>, <TipoMecanica.FCEV: 'FCEV'>, <TipoMecanica.GASOLINA: 'GASOLINA'>, <TipoMecanica.HEVD: 'HEVD'>, <TipoMecanica.HEVG: 'HEVG'>, <TipoMecanica.MHEVD: 'MHEVD'>, <TipoMecanica.MHEVG: 'MHEVG'>, <TipoMecanica.PHEVD: 'PHEVD'>, <TipoMecanica.PHEVG: 'PHEVG'>, <TipoMecanica.R

INFO:root:Filtro Económico: Rango de Cuota -> 198€/mes a 396€/mes
DEBUG:google.cloud.bigquery.opentelemetry_tracing:This service is instrumented using OpenTelemetry. OpenTelemetry or one of its components could not be imported; please add compatible versions of opentelemetry-api and opentelemetry-instrumentation packages in order to get BigQuery Tracing data.
DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None


--- 🧠 SQL Query Template Enviada a BigQuery ---

    WITH ScaledData AS (
        SELECT
            *,
            COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS estetica_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS premium_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS singular_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(altura_libre_suelo, 79.0) - 79.0, NULLIF(314.0 - 79.0, 0)), 0) AS altura_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(batalla, 1650.0) - 1650.0, NULLIF(4035.0 - 1650.0, 0)), 0) AS batalla_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(indice_altura_interior, 0.9) - 0.9, NULLIF(2.7 - 0.9, 0)), 0) AS indice_altura_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(ancho, 1410.0) - 1410.0, NULLIF(2164.0 - 1410.0, 0)), 0) AS ancho_scaled,
            COALESCE(SAFE_DIVIDE(COALESCE(fiabilidad, 1.0) - 1.0, NULLIF(10.0

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): bigquery.googleapis.com:443
DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/thecarmentor-mvp2/jobs?prettyPrint=false HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/thecarmentor-mvp2/queries/44032c2b-6fb3-4f54-9503-70874cc37f98?maxResults=0&location=EU&prettyPrint=false HTTP/1.1" 200 None
INFO:root:✅ BigQuery query ejecutada, 7 resultados obtenidos.
DEBUG:root:DEBUG (Buscar BQ) ► Llamando a buscar_coches_bq con k=7
INFO:root:INFO (Buscar BQ) ► 7 coches sanitizados y listos.
DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new

INFO (BQ Logger) ► Log para 'test_thread_notebook_1234' guardado en BQ.


--- 🏁 ANÁLISIS DEL RESULTADO FINAL ---

--- Mensaje de Texto (lo que ve el usuario) ---
================================== Ai Message ==================================

¡Listo! Basado en todo lo que hablamos, aquí tienes 7 coche(s) que podrían interesarte:

--- 🕵️ Datos Estructurados (lo que procesa el frontend) 🕵️ ---

Se encontraron 7 coches en el payload:


,name,specs,imageUrl,price,score,analysis
0,1. BMW Z4 Coupé 3.0si,"[GASOLINA, 2008, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im01610-1-bmw-z4.jpg,24.900€,44.92 pts,Análisis detallado de la recomendación pendiente de desarrollo.
1,2. Mercedes-Benz SLK 350,"[GASOLINA, 2007, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im00985-1-mercedes-slk-klasse.jpg,15.300€,42.16 pts,Análisis detallado de la recomendación pendiente de desarrollo.
2,3. Mercedes-Benz SLC 200,"[GASOLINA, 2016, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im03480-1-mercedes-slc-klasse.jpg,27.690€,41.99 pts,Análisis detallado de la recomendación pendiente de desarrollo.
3,4. Audi TT Coupé 1.8 TFSI,"[GASOLINA, 2008, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im01632-1-audi-tt.jpg,15.000€,41.39 pts,Análisis detallado de la recomendación pendiente de desarrollo.
4,5. Abarth 695C Rivale,"[GASOLINA, 2018, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im03748-1-fiat-500-abarth.jpg,27.999€,40.66 pts,Análisis detallado de la recomendación pendiente de desarrollo.
5,6. VW Scirocco 2.0 TSI BMT GTS,"[GASOLINA, 2016, FWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im03090-1-vw-scirocco.jpg,21.900€,40.01 pts,Análisis detallado de la recomendación pendiente de desarrollo.
6,7. Alfa Romeo Spider 2.0 Kat.,"[GASOLINA, 1990, RWD]",https://assets.adac.de/image/upload/v1/Autodatenbank/Fahrzeugbilder/im03583-1-alfa-romeo-spider.jpg,19.900€,39.09 pts,Análisis detallado de la recomendación pendiente de desarrollo.


In [3]:

import pandas as pd
coches_recomendados = current_state[('coches_recomendados')]

df_resultados = pd.DataFrame(coches_recomendados)
        
#  OPCIÓN 1: Imprimir el DataFrame directamente (Pandas usará las opciones de display)
# print("\n--- Vista DataFrame (Pandas Display Options) ---")
# display(df_resultados) # 'display()' suele dar mejor formato en notebooks que 'print(df)'

#OPCIÓN 2: Convertir a Markdown (como en tu nodo, pero para consola)
print("\n--- Vista Markdown ---") 
columnas_deseadas = [
    # Columnas principales
    'nombre', 'ID', 'marca', 'modelo', 'tipo_mecanica', 'plazas' ,'km_ocasion'  ,'ano_unidad', 'anos_vehiculo', 'tipo_carroceria' ,'reductoras', 'traccion' ,'score_total', 'puntuacion_base', 'ajustes_experto',
    'dbg_score_estetica', 'estetica', 'dbg_score_premium', 'premium' ,'dbg_score_singular', 'singular', 'dbg_score_altura_libre', 'altura_libre_suelo',
    'dbg_score_batalla', 'batalla', 'dbg_score_altura_interior','indice_altura_interior',  'dbg_score_ancho', 'ancho' ,'dbg_score_largo','largo' ,'dbg_score_devaluacion', 'devaluacion',
    'dbg_score_maletero_min', 'maletero_minimo','dbg_score_maletero_max', 'maletero_maximo' ,
    'dbg_score_bajo_peso', 'peso' , 
    'dbg_score_par_remolque', 'par' ,
    'dbg_score_remolque_cf', 'capacidad_remolque_con_freno',
    'dbg_score_remolque_sf',  'capacidad_remolque_sin_freno',
    'dbg_score_menor_superficie', 'superficie_planta',
    'dbg_score_menor_giro', 'diametro_giro' , 
    'dbg_score_menor_largo', 'largo',
    'dbg_score_menor_ancho', 'ancho',
    'dbg_score_menor_alto', 'alto',
    'dbg_score_deportividad', 'deportividad',
    'dbg_score_menor_rel_peso_pot', 'relacion_peso_potencia',
    'dbg_score_potencia', 'potencia_maxima',
    'dbg_score_par_deportivo', 'par',
    'dbg_score_menor_aceleracion' ,'aceleracion_0_100',
    'dbg_score_autonomia_max', 'autonomia_uso_maxima', 
    'dbg_score_autonomia_2nd', 'autonomia_uso_2nd_drive',
    'dbg_score_menor_t_carga', 'tiempo_carga_min',
    'dbg_score_pot_carga_ac',  'potencia_maxima_carga_ac',
    'dbg_score_pot_carga_dc',  'potencia_maxima_carga_dc',
    'dbg_pen_km_extremo' ,
    'dbg_pen_puertas' , 'dbg_pen_low_cost_comodidad' , 'dbg_pen_deportividad_comodidad' ,
    'dbg_pen_antiguedad' , 'dbg_ajuste_distintivo' , 'dbg_bonus_ocasion_ambiental' , 'dbg_ajuste_zbe' ,
    'dbg_pen_bev_reev_avent_ocas' , 'dbg_pen_phev_avent_ocas' , 'dbg_pen_electrif_avent_extr' ,
    'dbg_bonus_car_montana' , 'dbg_bonus_car_comercial' , 'dbg_bonus_car_pasajeros' , 'dbg_pen_car_no_aventura' ,
    'dbg_bonus_suv_avent_ocas' , 'dbg_bonus_tt_avent_extr' , 'dbg_bonus_pickup_avent_extr' , 
    'dbg_ajuste_objetos_especiales' , 'dbg_bonus_car_confort' , 'dbg_bonus_ocasion_uso_ocas' ,
    'dbg_pen_electrif_uso_ocas' , 'dbg_bonus_bev_uso_definido' , 'dbg_pen_phev_uso_intensivo' ,
    'dbg_bonus_punto_carga' , 'dbg_ajuste_awd_aventura' , 'dbg_bonus_awd_nieve' , 'dbg_bonus_awd_montana' ,
    'dbg_bonus_reductoras' , 'dbg_ajuste_diesel_ciudad' , 'dbg_ajuste_km_anuales', 
    'dbg_bonus_seguridad', 'seguridad', 
    'dbg_bonus_fiabilidad' ,'fiabilidad', 
    'dbg_bonus_durabilidad' , 'durabilidad', 
    'indice_consumo_energia','dbg_bonus_bajo_consumo', 'costes_de_uso'  , 'dbg_bonus_coste_uso' ,  
    'dbg_bonus_coste_mantenimiento' ,'costes_mantenimiento' ,'dbg_pen_antiguedad_general','dbg_pen_tamano_contextual', 
    'dbg_bonus_lifestyle' ,  'dbg_ajuste_deportividad_lifestyle', 'dbg_pen_bev_lifestyle' ,'dbg_ajuste_maletero_personal' , 'dbg_bonus_coche_ciudad' , 'dbg_bonus_coche_ciudad_2'
]
columnas_existentes = [col for col in columnas_deseadas if col in df_resultados.columns]
if columnas_existentes:
            if 'score_total' in df_resultados.columns: # Formatear score para legibilidad
                df_resultados['score_total'] = df_resultados['score_total'].apply(lambda x: f"{x:.4f}" if isinstance(x, float) else x)
            print(df_resultados[columnas_existentes].to_markdown(index=False))
else:
            print("WARN: No se encontraron columnas esperadas para mostrar en formato tabla.")
            print("Resultados crudos (lista de dicts):", coches_recomendados) 


--- Vista Markdown ---
| nombre                      |     ID | marca         | modelo      | tipo_mecanica   |   plazas |   km_ocasion |   ano_unidad |   anos_vehiculo | tipo_carroceria   | reductoras   | traccion   |   score_total |   puntuacion_base |   ajustes_experto |   dbg_score_estetica |   estetica |   dbg_score_premium |   premium |   dbg_score_singular |   singular |   dbg_score_altura_libre |   altura_libre_suelo |   dbg_score_batalla |   batalla |   dbg_score_altura_interior |   indice_altura_interior |   dbg_score_ancho |   ancho |   dbg_score_largo |   largo |   dbg_score_devaluacion |   devaluacion |   dbg_score_maletero_min |   maletero_minimo |   dbg_score_maletero_max |   maletero_maximo |   dbg_score_bajo_peso |   peso |   dbg_score_par_remolque |   par |   dbg_score_remolque_cf |   capacidad_remolque_con_freno |   dbg_score_remolque_sf |   capacidad_remolque_sin_freno |   dbg_score_menor_superficie |   superficie_planta |   dbg_score_menor_giro |   diametro_giro |

In [ ]:

print('flag_es_conductor_urbano:' , current_state['flag_es_conductor_urbano'])
print('-----------------------') 
print('flag_penalizar_tamano_no_compacto:' , current_state['flag_penalizar_tamano_no_compacto'])
print('-----------------------') 
print('flag_bonus_singularidad_lifestyle:' , current_state['flag_bonus_singularidad_lifestyle']) 
print('-----------------------') 
print('flag_deportividad_lifestyle:' , current_state['flag_deportividad_lifestyle'])
print('-----------------------') 
print('-----------------------') 
print('flag_ajuste_maletero_personal:' , current_state['flag_ajuste_maletero_personal'])
print('-----------------------') 
print('flag_coche_ciudad_perfil:' , current_state['flag_coche_ciudad_perfil'])
print('-----------------------') 
print('flag_coche_ciudad_2_perfil:' , current_state['flag_coche_ciudad_2_perfil'])
print('-----------------------') 
print('flag_ajuste_maletero_personal:' , current_state['flag_ajuste_maletero_personal'])



In [ ]:
print(type(current_state['flag_bonus_fiab_dur_critico']))


In [ ]:
print(current_state.keys())

In [ ]:
print(current_state['codigo_postal_usuario'])
print('-----------------------')
print(current_state['info_clima_usuario'])
print('-----------------------')
print(current_state['preferencias_usuario'])
print('-----------------------')
print(current_state['filtros_inferidos'])
print('-----------------------')
print(current_state['info_pasajeros'])
print('-----------------------')
print(current_state['economia'])
print('-----------------------')
print(current_state['pesos'])
print('-----------------------')
print("info_pasajeros:",current_state['info_pasajeros'])
print('-----------------------')

In [ ]:
print("flag_penalizar_deportividad_comodidad:", current_state['flag_penalizar_deportividad_comodidad'])
print('-----------------------')
print("flag_penalizar_low_cost_comodidad:", current_state['flag_penalizar_low_cost_comodidad'])
print('-----------------------')
print("penalizar_puertas_bajas:" , current_state['penalizar_puertas_bajas'])
print('-----------------------')
print("flag_penalizar_antiguo_por_tecnologia:" , current_state['flag_penalizar_antiguo_por_tecnologia'])
print('-----------------------')
print("es municipio_zbe: " , current_state['es_municipio_zbe'])
print('-----------------------')
print("penalizar_electrificados_aventura_extrema: " , current_state['penalizar_electrificados_aventura_extrema'])
print('-----------------------')
print('favorecer_carroceria_montana:', current_state['favorecer_carroceria_montana'])
print('-----------------------')
print('favorecer_carroceria_comercial', current_state['favorecer_carroceria_comercial'])
print('-----------------------')
print('favorecer_carroceria_pasajeros_pro:', current_state['favorecer_carroceria_pasajeros_pro'])
print('-----------------------')
print('aplicar_logica_objetos_especiales:', current_state['aplicar_logica_objetos_especiales'])#OK 
print('-----------------------')
print('favorecer_carroceria_confort:', current_state['favorecer_carroceria_confort']) #OK
print('-----------------------')
print('flag_logica_uso_ocasional:' , current_state['flag_logica_uso_ocasional'])
print('-----------------------')
print('flag_favorecer_bev_uso_definido:' , current_state['flag_favorecer_bev_uso_definido'])
print('-----------------------')
print('flag_penalizar_phev_uso_intensivo:' , current_state['flag_penalizar_phev_uso_intensivo'])
print('-----------------------')
print('flag_favorecer_electrificados_por_punto_carga:' , current_state['flag_favorecer_electrificados_por_punto_carga'])
print('-----------------------')
print('km_anuales_estimados:' , current_state['km_anuales_estimados'])

In [ ]:

print('desfavorecer_carroceria_no_aventura:', current_state['desfavorecer_carroceria_no_aventura']) #Aventura nula y no en montaña desfavorece PICKUP/TODOTERRENO
print('-----------------------')
print('favorecer_suv_aventura_ocasional:', current_state['favorecer_suv_aventura_ocasional']) #OK
print('-----------------------')
print('favorecer_pickup_todoterreno_aventura_extrema:', current_state['favorecer_pickup_todoterreno_aventura_extrema'])
print('-----------------------')
print("penalizar_bev_reev_aventura_ocasional: " , current_state['penalizar_bev_reev_aventura_ocasional'])
print('-----------------------')
print("penalizar_phev_aventura_ocasional: " , current_state['penalizar_phev_aventura_ocasional'])
print('-----------------------')
print('penalizar_awd_ninguna_aventura:' , current_state['penalizar_awd_ninguna_aventura'])
print('-----------------------')
print('favorecer_awd_aventura_ocasional:' , current_state['favorecer_awd_aventura_ocasional'])
print('-----------------------')
print('favorecer_awd_aventura_extrema:' , current_state['favorecer_awd_aventura_extrema'])
print('-----------------------')
print('flag_bonus_awd_nieve:' , current_state['flag_bonus_awd_nieve'])
print('-----------------------')
print('flag_bonus_awd_montana:' , current_state['flag_bonus_awd_montana'])
print('-----------------------')
print('flag_logica_reductoras_aventura:' , current_state['flag_logica_reductoras_aventura'])
print('-----------------------')
print('flag_logica_diesel_ciudad:' , current_state['flag_logica_diesel_ciudad'])
print('-----------------------') 
print('flag_bonus_awd_clima_adverso:' , current_state['flag_bonus_awd_clima_adverso']) 
print('-----------------------') 
print('flag_bonus_seguridad_critico:' , current_state['flag_bonus_seguridad_critico'])
print('-----------------------') 
print('flag_bonus_seguridad_fuerte:' , current_state['flag_bonus_seguridad_fuerte'])
print('-----------------------')
print("aplicar_logica_distintivo_ambiental:" ,  current_state['aplicar_logica_distintivo_ambiental'])
print('-----------------------') 
print('flag_bonus_fiab_dur_critico' , current_state['flag_bonus_fiab_dur_critico'])
print('-----------------------') 
print('flag_bonus_fiab_dur_fuerte:' , current_state['flag_bonus_fiab_dur_fuerte'])
print('-----------------------')
print('flag_bonus_costes_critico') , current_state['flag_bonus_costes_critico']

In [ ]:
print(current_state['coches_recomendados']('columnas_deseadas'))

In [ ]:
df_resultados[columnas_existentes].to_csv(
    "resultados_filtrados.csv",
    index=False,
    sep=";",
    float_format="%.4f",
    encoding="utf-8"
)


# df_resultados.to_csv(
#     "resultados_filtradosV2.csv",
#     index=False,
#     sep=";",
#     float_format="%.4f",
#     encoding="utf-8"
# )
